import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
for i in range (1,6):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=ECG5000(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,6):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
print('shape ecg=',np.shape(ecg))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (500, 141)
first_test_shape= (4500, 141)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [292 177  10  19   2]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -5.7976446
first_train_sample=
 [ 1.         -0.11252183 -2.8272038  -3.7738969  -4.3497511  -4.376041
 -3.4749863  -2.1814082  -1.8182865  -1.2505219  -0.47749208 -0.36380791
 -0.49195659 -0.42185509 -0.30920086 -0.4959387  -0.34211867 -0.35533627
 -0.36791303 -0.31650279 -0.41237405 -0.47167181 -0.41345783 -0.36461703
 -0.44929829 -0.47141866 -0.42477658 -0.46251673 -0.55247236 -0.47537519
 -0.6942     -0.7018681  -0.59381178 -0.66068415 -0.71383066 -0.76980688
 -0.67228161 -0.65367605 -0.63940562 -0.55930228 -0.59167032 -0.49322332
 -0.46305183 -0.30164382 -0.23273401 -0.12505488 -0.15394314 -0.0243574
 -0.06560876  0.03499926  0.06193522  0.07119542  0.12392505  0.10312371
  0.22522849  0.12868305  0.30248315  0.25727621  0.19635161  0.17938297
  0.24472863  0.34121687  0.32820441 

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [3]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,6):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,6):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 292	2 177	3 10	4 19	5 2	
 max = 292


In [4]:
#clses=3                                   #تعیین میزان افزایش نمونه (تولید داده)
mx_wndws=int(mx_wndws+6)  #mx_wndws*0.3  # =classes-1 برای بررسی الگوریتم تعداد کمی کلاس آزمایش شد
print(mx_wndws)

298


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [5]:
methd=40
sort=-5          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
dp_slct_dstrb=1  #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm

for i in range (1,6):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+str(methd)]=np.empty((0,smpl_rte+1), float) 
for i in range (1,6):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    print("vlum_win=",vlum_win)
    vars()['ecg_aug'+str(i)+str(methd)]=np.append(vars()['ecg_aug'+str(i)+str(methd)],srtd_data(methd,i,vlum_win,dp_slct_dstrb,sort),axis=0)
    #print('aug shape=',np.shape(scl_rndm(ecg,i,vlum_win,mag=.1)))
    print('generated data shape for class ', i, ' = ', np.shape(vars()['ecg_aug'+str(i)+str(methd)]))

vlum_win= 6
final shape=  (6, 487)
generated data shape for class  1  =  (6, 481)
vlum_win= 121
final shape=  (121, 487)
generated data shape for class  2  =  (121, 481)
vlum_win= 288
final shape=  (288, 487)
generated data shape for class  3  =  (288, 481)
vlum_win= 279
final shape=  (279, 487)
generated data shape for class  4  =  (279, 481)
vlum_win= 296
final shape=  (296, 487)
generated data shape for class  5  =  (296, 481)


In [6]:
srtd_data(methd,i,vlum_win,dp_slct_dstrb,sort)

final shape=  (296, 487)


array([[-1.18371468, -1.80712789, -2.12235241, ..., -0.32307538,
        -0.67957645,  5.        ],
       [ 0.27883586,  0.17452148, -0.31904061, ...,  0.23056262,
         0.42243063,  5.        ],
       [-0.50830387, -1.48186337, -2.43154203, ..., -0.34620698,
        -0.59865929,  5.        ],
       ...,
       [ 0.00863786,  0.1098167 , -0.13911286, ...,  0.1258963 ,
         0.23124683,  5.        ],
       [ 0.1945143 ,  0.41674907, -0.60907631, ...,  0.16604401,
         0.45318923,  5.        ],
       [ 0.31422293, -0.0724321 , -0.25043005, ...,  0.19327331,
         0.16389301,  5.        ]])

In [7]:
vlum_win

296

In [8]:
           #add augmented data to base data
xtrain=np.empty((0,smpl_rte+1), float) 
for i in range (1,6):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    xtrain=np.append(xtrain,vars()['ecg_aug'+str(i)+str(methd)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (298, 481)
Up to class  2 train shape =  (596, 481)
Up to class  3 train shape =  (894, 481)
Up to class  4 train shape =  (1192, 481)
Up to class  5 train shape =  (1490, 481)


In [9]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[  0 298 298 298 298 298]


In [10]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,6):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [11]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [12]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,6):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [13]:
#print(np.max(wndws_tst))
for i in range (1,6):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,6):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  5.398327935600255
min magnitude class 1  =  -8.366738588591014
after normalizing >>
max magnitude class 1  =  1.1995570037668069
min magnitude class 1  =  -1.368121323421097

 cls 2  >> 
max magnitude class 2  =  7.557156182756754
min magnitude class 2  =  -6.623263697325152
after normalizing >>
max magnitude class 2  =  1.6022558568819
min magnitude class 2  =  -1.0429007598711757

 cls 3  >> 
max magnitude class 3  =  3.427621598045422
min magnitude class 3  =  -6.827926378639614
after normalizing >>
max magnitude class 3  =  0.8319496301810339
min magnitude class 3  =  -1.0810776862428544

 cls 4  >> 
max magnitude class 4  =  3.033042703743151
min magnitude class 4  =  -5.658319868425961
after normalizing >>
max magnitude class 4  =  0.7583465223801995
min magnitude class 4  =  -0.8629041465555902

 cls 5  >> 
max magnitude class 5  =  2.4648784169429363
min magnitude class 5  =  -4.7540569
after normalizing >>
max magnitude class 5  =  0.6523

In [14]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.368121323421097
max tst = 1.6022558568819


In [15]:
len(wndws_test)

4500

In [16]:
np.shape(wndws_test)

(4500, 481)

In [17]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [18]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [19]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [20]:
np.shape(wndws1)

(292, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [21]:
Xvalid=np.array(Xtest)
yvalid=np.array(ytest)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [22]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtest)
y_valid=np.array(ytest)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (1490, 480)
y_train =>  (1490,)
X_test  =>  (4500, 480)
y_test  =>  (4500,)
X_valid  =>  (4500, 480)
y_valid  =>  (4500,)


In [23]:
y_train=np.array(ytrain)
np.min(y_train)

1

In [24]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [25]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 30
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
50/50 [==============================] - 4s 21ms/step - loss: 0.7813 - accuracy: 0.6980 - val_loss: 0.6375 - val_accuracy: 0.7876
Epoch 2/1000
50/50 [==============================] - 1s 16ms/step - loss: 0.2386 - accuracy: 0.9221 - val_loss: 0.4792 - val_accuracy: 0.8747
Epoch 3/1000
50/50 [==============================] - 1s 15ms/step - loss: 0.1560 - accuracy: 0.9523 - val_loss: 0.3551 - val_accuracy: 0.9238
Epoch 4/1000
50/50 [==============================] - 1s 16ms/step - loss: 0.1614 - accuracy: 0.9591 - val_loss: 0.3626 - val_accuracy: 0.9287
Epoch 5/1000
50/50 [==============================] - 1s 16ms/step - loss: 0.1096 - accuracy: 0.9638 - val_loss: 0.3628 - val_accuracy: 0.9269
Epoch 6/1000
50/50 [==============================] - 1s 16ms/step - loss: 0.1065 - accuracy: 0.9651 - val_loss: 0.6395 - val_accuracy: 0.8871
Epoch 7/1000
50/50 [==============================] - 1s 16ms/step - loss: 0.1012 - accuracy: 0.9671 - val_loss: 0.5464 - val_accuracy: 0.9176

 dense_1 (Dense)             (None, 6)                 12006     
                                                                 
Total params: 2,240,886
Trainable params: 2,240,886
Non-trainable params: 0
_________________________________________________________________
Accuracy=  0.941777765750885


In [26]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [27]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
print(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

141/141 [==============================] - 0s 2ms/step
[[2606   13    2    3    3]
 [  21 1531   20   18    0]
 [  18   29   31    5    3]
 [   6   95    4   68    2]
 [   9    8    3    0    2]]
              precision    recall  f1-score   support

           1       0.98      0.99      0.99      2627
           2       0.91      0.96      0.94      1590
           3       0.52      0.36      0.42        86
           4       0.72      0.39      0.51       175
           5       0.20      0.09      0.13        22

    accuracy                           0.94      4500
   macro avg       0.67      0.56      0.60      4500
weighted avg       0.93      0.94      0.94      4500



In [28]:
print(np.bincount(y_train)[1:])

[298 298 298 298 298]
